# Lab03: Web Crawler (Continue) & Information Retrieval.

- MSSV: 1712282
- Họ và tên: LÊ ĐOÀN CÔNG ẢNH

## Yêu cầu bài tập

**Cách làm bài**


Bạn sẽ làm trực tiếp trên file notebook này; từ `TODO` cho biết những phần mà bạn cần phải làm.

Bạn có thể thảo luận ý tưởng cũng như tham khảo các tài liệu, nhưng *code và bài làm phải là của bạn*. 

Nếu vi phạm thì sẽ bị 0 điểm cho bài tập này.

**Cách nộp bài**

Trước khi nộp bài, rerun lại notebook (`Kernel` -> `Restart & Run All`).

Sau đó, tạo thư mục có tên `MSSV` của bạn (vd, nếu bạn có MSSV là 1234567 thì bạn đặt tên thư mục là `1234567`) Chép file notebook, file `t_data.txt` và file `raw_data` của các bạn (nếu file này kích thước lớn các bạn có thể chép link vào `link_data.txt`), nén thư mục `MSSV` này lại và nộp trên moodle.

**Nội dung bài tập**

Cài đặt một web crawler để thu thập dữ liệu từ: https://en.wikipedia.org/wiki/Web_mining.

## Nội dung bài tập

Cài đặt một Web crawler đơn giản bắt đầu từ URL: https://en.wikipedia.org/wiki/Web_mining, tìm liên kết và thu thập dữ liệu trong HTML tại URL này sau đó lặp lại với các URL vừa tìm được.


In [1]:
import requests
import re
from bs4 import BeautifulSoup
from bs4.element import Comment
import string
import pickle

## 1. Thu thập đường dẫn

- Robot.txt:  https://en.wikipedia.org/robots.txt
- **Bước 1**: Thu thập đường dẫn từ https://en.wikipedia.org/wiki/Web_mining. Lưu trữ vào một danh sách `url_list`. 
- **Bước 2**: Lặp lại bước 1 cho các đường dẫn trong `url_list` (**lưu ý:** kiểm tra các đường dẫn vừa thu được đã nằm trong `url_list` hay không?). Dừng khi đã thu thập được 200 URLs.

In [2]:
def get_urls(url):
    r = requests.get(url)
    # TODO
    # Lấy các url nằm trong trang web của url này, lưu lại vào biến urls
    regex = "(?:(?:https?|ftp):\/\/)[\w/\-?=%.]+\.[\w/\-&?=%.]+"
    data = r.text
    urls = re.findall(regex, data)

    # remove duplicates
    urls = list(set(urls))
    return urls


def get_urls_recursive(start_url, limit):
    urls = [start_url]
    for url in urls:
        # TODO
        # Lấy các url nằm trong trang web của url này, lưu lại vào biến new_urls
        # Với mỗi url mới trong new_urls:
        #   Nếu nó chưa nằm trong urls thì thêm nó vô  
        # Nếu kích thước của urls vượt quá limit thì dừng và xóa phần dư thừa
        new_urls = get_urls(url)
        for i in new_urls:
            if not i in urls:
                urls.append(i)
        
        if (len(urls) >= limit):
            urls = urls[0:limit]
            break
    return urls
url_list = get_urls_recursive('https://en.wikipedia.org/wiki/Web_mining', 50)

# check url_list

for url_index, url in enumerate(url_list, 1):
    print(str(url_index).ljust(5) + ' : ' + url)

1     : https://en.wikipedia.org/wiki/Web_mining
2     : https://pt.wikipedia.org/wiki/Minera%C3%A7%C3%A3o_da_web
3     : http://webmining.spd.louisville.edu/wp-content/uploads/2014/05/Nasraoui_WebKDD03_web_recomm.pdf
4     : https://www.mediawiki.org/
5     : https://archive.org/details/webminingapplica0000scim/page/282
6     : https://hr.wikipedia.org/wiki/Dubinsko_pretra%C5%BEivanje_interneta
7     : http://alexandria.tue.nl/repository/freearticles/612259.pdf
8     : http://www.chris-kimble.com/Publications/Documents/Ting_2005b.pdf
9     : https://archive.org/details/webinformationsy00ngua
10    : https://doi.org/10.1007/3-540-48298-9_32
11    : https://www.wikidata.org/wiki/Special
12    : https://en.wikipedia.org/w/index.php?title=Web_mining&amp
13    : https://eu.wikipedia.org/wiki/Web-meatzaritza
14    : http://csdl2.computer.org/persagen/DLAbsToc.jsp?resourcePath=/dl/proceedings/wi/&amp
15    : https://www.mediawiki.org/wiki/Special
16    : http://webmining.spd.louisville.edu/w

## 2. Thu thập dữ liệu
Thu thập dữ liệu từ `url_list`. Lưu trữ dữ liệu thu được vào dictionary data với keys là các từ, values gồm 2 phần tử: 
- `url_idx_list` với $idx \in \left[0,200\right) \cap \mathbb{N}$
- `frequency` 
    
Ví dụ: `data['at']=[url_idx_list,frequency]`:
- `url_idx_list`: danh sách các url mà trong dữ liệu của chúng (html document) chứa từ "at". 
- `frequency`: tần suất xuất hiện (số lần xuất hiện) của từ `at` trong dữ liệu của **tất cả đường dẫn thu được**.   

In [3]:
def text_filter(element):
    # TODO
    # Cài đặt lại như Lab02
    if element.parent.name in ['style', 'title', 'script', 'head', '[document]', 'class', 'a', 'li']:
        return False
    elif isinstance(element, Comment):
        '''Opinion mining?'''
        return False
    elif re.match(r"[\s\r\n]+",str(element)): 
        '''space, return, endline'''
        return False
    return True

def wordList(url):
    # TODO
    # Cài đặt lại như Lab02
    r = requests.get(url)
    soup = BeautifulSoup(r.content, "html.parser",from_encoding="UTF-8")
    text = soup.findAll(text=True)
    filtered_text = list(filter(text_filter, text)) # list của các chuỗi
    word_list = []

    # TODO:
    # Với mỗi chuỗi trong filtered_text:
    #   Thay thế các dấu câu thành khoảng trắng (gợi ý: danh sách các dấu câu: string.punctuation; thay thế: .replace(...))
    #   Tách chuỗi bởi khoảng trắng (.split(...))
    #   Thêm các từ vừa được tách ra vào word_list
    
    # remove all punctuation in filered_text
    for i in range(len(filtered_text)):
        for ele in filtered_text[i]:
            if ele in string.punctuation:
                filtered_text[i] = filtered_text[i].replace(ele, " ")

            # remove '\n' from filtered_text
            filtered_text[i] = filtered_text[i].replace('\n', " ")
            filtered_text[i] = filtered_text[i].replace('–', " ")
            filtered_text[i] = filtered_text[i].replace('\xa0', " ")
    
    # split by ' ' from filtered_text
    for i in filtered_text:
        word_list += i.split(' ')

    # remove all element '' in word_list
    word_list = list(filter(lambda a: a != '', word_list))
    return word_list

def read_url(url, url_idx, data):
    # TODO
    # Cài đặt lại như Lab02
    word_list = wordList(url)
    # TODO
    # Với mỗi từ w trong word_list:
    #   Nếu w chưa có trong data thì khởi tạo data[w] = [[url_idx], 1]
    #   Ngược lại thì thêm url_idx vào data[w][0] (nếu chưa có) và tăng data[w][1] lên 1 đơn vị
    
    for w in word_list:
        if w in data:
            data[w][1] += 1
            if not url_idx in data[w][0]:
                data[w][0].append(url_idx)
        else:
            data[w] = [[url_idx], 1]

In [4]:
data = {}
for url_index, url in enumerate(url_list, 1):
    print('Executing is '+ str(url_index))
    read_url(url, url_index, data)

data

Executing is 1
Executing is 2
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Executing is 3
Executing is 4
Executing is 5
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Executing is 6
Executing is 7
Executing is 8
Executing is 9
Executing is 10


], 1],
 'might': [[1], 3],
 'use': [[1, 3], 4],
 'totally': [[1], 1],
 'purposes': [[1], 1],
 'violates': [[1], 1],
 'user': [[1, 8], 7],
 'interests': [[1], 1],
 'growing': [[1], 1],
 'trend': [[1], 2],
 'selling': [[1], 1],
 'personal': [[1], 2],
 'commodity': [[1], 1],
 'encourages': [[1], 1],
 'owners': [[1], 1],
 'increased': [[1], 1],
 'amount': [[1], 1],
 'being': [[1], 2],
 'captured': [[1], 1],
 'traded': [[1], 1],
 'increasing': [[1], 1],
 'likeliness': [[1], 1],
 'one': [[1], 2],
 'privacy': [[1], 1],
 'invaded': [[1], 1],
 'buy': [[1], 1],
 'obliged': [[1], 1],
 'anonymous': [[1], 1],
 'these': [[1], 1],
 'authors': [[1], 1],
 'any': [[1], 2],
 'release': [[1], 3],
 'patterns': [[1, 8], 3],
 'They': [[1], 1],
 'legally': [[1], 1],
 'responsible': [[1], 1],
 'contents': [[1], 1],
 'inaccuracies': [[1], 1],
 'result': [[1], 2],
 'serious': [[1], 1],
 'lawsuits': [[1], 1],
 'there': [[1], 1],
 'no': [[1, 5], 2],
 'law': [[1], 1],
 'preventing': [[1], 1],
 'them': [[1], 1],
 't

## 3. Tiền xử lý
Loại bỏ các item trong data mà key là các stopword.

**Ngữ liệu:** 

In [35]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
english_stopwords = stopwords.words('english')
print(english_stopwords)

[nltk_data] Downloading package stopwords to /Users/macos/nltk_data...
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'so

In [ ]:
# TODO
# Loại bỏ các key của biến data mà nằm trong danh sách english_stopwords

## 4. Lưu trữ và biểu diễn dữ liệu
Sử dụng pickle lưu lại data với tên file raw_data.
### 4.1 Cơ sở dữ liệu giao tác:
Thông thường, các cơ sở dữ liệu giao tác được lưu trong flat files (các tập phẳng) thay vì trong một hệ cơ sở dữ liệu. Các item là các số nguyên không âm, mỗi giao tác tương ứng với một dòng các số nguyên phân tách nhau bằng khoảng trắng.
Ví dụ:

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 

30 31 32 

33 34 35 

36 37 38 39 40 41 42 43 44 45 46 

38 39 47 48 

38 39 48 49 50 51 52 53 54 55 56 57 58 

32 41 59 60 61 62 

3 39 48 

63 64 65 66 67 68 

32 69 

48 70 71 72 

39 73 74 75 76 77 78 79 

36 38 39 41 48 79 80 81 

82 83 84 

41 85 86 87 88 

39 48 89 90 91 92 93 94 95 96 97 98 99 100 101 

36 38 39 48 89 

39 41 102 103 104 105 106 107 108 

38 39 41 109 110 

39 111 112 113 114 115 116 117 118 

119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 

48 134 135 136 

39 48 137 138 139 140 141 142 143 144 145 146 147 148 149 

39 150 151 152 

38 39 56 153 154 155 

In [ ]:
with open('raw_data', 'wb') as f:
    # TODO

### 4.2 Xuất dataset
Lưu một cơ sở dữ liệu giao tác (transactional database) vào file t_data.txt: 
- Các item tương ứng với url_idx
- Mỗi transaction tương ứng với một từ.

In [ ]:
with open('t_data.txt', 'w') as f:
    for word, (url_list, freq) in data.items():
        print(*url_list, file=f)

## 5. Truy vấn and, or, not
Ví dụ: 
- Truy vấn `and` câu `web mining`: trả về đường dẫn tới các trang web có cả 2 từ web và từ mining. 
- Truy vấn `or` câu `web mining`: trả về đường dẫn tới các trang web có từ web hoặc từ mining.
- Truy vấn `not` câu `web mining`: trả về đường dẫn tới các trang không có cả từ web và từ mining.

*GỢI Ý: TÁCH CÂU TRUY VẤN THÀNH CÁC TỪ TƯƠNG TỰ PHƯƠNG PHÁP LÀM Ở LAB02.*

In [ ]:
ret=[]
def andRetrieval(ret, sentence):
    '''Parameters
    -----------------------
    ret: url_list
    sentence: query'''
    # TODO
    ### split sentence (separator ' ') into terms
    ### find urls have all terms: urls
    ### if len(ret)==0:  return urls
    ### else update ret with urls: intersection of ret and urls 


def orRetrieval(ret, sentence):
    '''Parameters
    -----------------------
    ret: url_list
    sentence: query'''
    # TODO
    ### split sentence (separator ' ') into terms
    ### find urls have all terms: urls
    ### find urls have at least 1 term: urls
    ### update ret with urls: extend ret with urls

def notRetrieval(ret, sentence):
    '''Parameters
    -----------------------
    ret: url_list
    sentence: query'''
    # TODO
    ### split sentence (separator ' ') into terms
    ### find urls have at least 1 term: urls
    ### update ret with urls: remove urls from ret 

In [ ]:
print(andRetrieval([], 'web mining'))
print(orRetrieval([], 'web mining'))
print(notRetrieval(url_list, 'web mining'))